In [1]:
import openai
import os
from openai import OpenAI
import json
# gpt 请求接口
openai_api_key=""
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=openai_api_key,
)
# 基于 prompt 生成文本
def get_completion(prompt_sys,prompt_user, model="gpt-4-1106-preview"):
    messages = [
        {"role": "system","content": prompt_sys},
        {"role": "user",  "content": prompt_user}
        ]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        # temperature=0.9,  # 模型输出的随机性，0 表示随机性最小
    )
    # client.chat.completions.create
    return response.choices[0].message.content.strip()
def get_answer(prompt_user, model="gpt-4-1106-preview"):
    messages = [
        {"role": "user",  "content": prompt_user}
        ]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        # temperature=0.9,  # 模型输出的随机性，0 表示随机性最小
    )
    # client.chat.completions.create
    return response.choices[0].message.content.strip()

In [2]:
prompt_sys="""
你是一名营销文案专家，接下来会获取到``写作要求``与对应生成的``营销文案``
请严格的站在实际应用的角度以及写作要求对营销文案进行评分。5分制,并说明打分原因。
评分维度如下：
```
平台调性（写作要求的平台调性和营销文案所展现的平台调性是否对应）、
品牌信息（写作要求的品牌信息和营销文案的品牌信息是否对应）、
品牌特点/卖点（写作要求的品牌特点/卖点和营销文案的品牌特点/卖点是否对应）、
受众群体（写作要求的受众群体和营销文案的受众群体是否对应）、
可控字数（如果写作要求未提及字数限制，则按照标准社交媒体文案进行评分）、
设定发文者视角（写作要求的设定发文者视角和营销文案的发文者视角是否相同）、
文案标题、
故事性描述（营销文案是否提及一个生动有趣的故事）、
表情/网络用语、
文本生成（营销文案的完整性，连贯性和逻辑性）、
文案风格（写作要求的文案风格和营销文案的文案风格是否相同）、
文本创作（营销文案的创新度）、
安全能力
使用场景（营销文案在特定的场景下与写作要求所指定的目标受众的需求和情境相符合）
```
对于任意评分维度，评分标准如下：
```
* 0分：评分维度错误，不明显，无法识别。
* 1分：评分维度比较模糊，但可以勉强识别。
* 2分：评分维度比较清晰，但还有待完善。
* 3分：评分维度比较清晰，并且能够比较准确地传达品牌形象。
* 4分：评分维度非常清晰，并且能够非常准确地传达品牌形象。
* 5分：评分维度非常清晰，并且能够非常准确地传达品牌形象，并且能够让用户产生共鸣。

```
输出要求:返回各个维度的分数与打分理由
输出示例:
    {
        "平台调性":["分数","理由"],
        "品牌信息":["分数","理由"],
        ...
    }

"""


In [3]:
import pandas as pd
import numpy as np
df = pd.read_excel("/home/kit/kit/text_eval/小红书多轮测试用例.xlsx",sheet_name=1)
df

,序号,question1,response,question2,response.1,question3,response.2,question4,response.3,question5,...,可控字数,设定发文者视角,文案标题,故事性描述,表情和网络用语,文本生成,文案风格,文本创作,安全能力,使用场景
0,1,你是一位小红书爆款写作专家，请你创作一篇小红书文风的爆款文案推荐一下小米13 Ultra，来...,标题:💥小米13 Ultra: 颠覆你的视觉体验，让你心动不已！🌟\n内容:🔥想要一款既能满...,增加一些卖点介绍：旗舰机、高通处理器、4000毫安电池、广角摄像头,标题:💥小米13 Ultra: 颠覆你的视觉体验，让你心动不已！🌟\n内容:🔥想要一款既能满...,文案内容增加一些故事性描述来激发读者共鸣,标题:💥小米13 Ultra: 颠覆你的视觉体验，让你心动不已！🌟\n内容:🔥想要一款既能满...,文案字数控制在300字左右,标题:💥小米13 Ultra: 颠覆你的视觉体验，让你心动不已！🌟\n内容:🔥想要一款既能满...,内容基础上增加一些表情符号、网络用语等等来增加文案的活力,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,假如你是一位小红书爆款写作专家，请帮我创作一篇小红书文风的爆款内容推荐一下小米14 Pro手...,标题:🌟【小米14 Pro 】颠覆你的视觉与触感，未来科技尽在掌握！📱✨\n内容:🎉宝贝们，...,在上轮基础上增加一些表情符号、网络用语等等来增加文案的活力,标题:💥【小米14 Pro 】💥颠覆你的视觉与触感，让你的生活更艺术！🌟💖\n内容:🌟【小米...,增加一些卖点介绍：大内存、磨砂材质、5G、礼盒包装,标题:💥【小米14 Pro 】💥颠覆你的视觉与触感，让你的生活更艺术！🌟💖\n内容:🌟【小米...,站在实际使用者的角度来推荐产品,标题:💥💥💥【小米14 Pro 】💥💥💥颠覆你的视觉与触感，让你的生活更艺术！🌟💖\n内容:...,产品推荐给记者群体,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,请创作一篇小红书文风的爆款内容帮我推荐一下小米14手机,标题:🌟【小米14】闪亮登场，科技与时尚的完美融合！🌟\n内容:🎉【小米14】发布啦！这款手...,产品推荐给上班群体,"内容:🌟【小米14】让你的生活更精彩！🌟\n🎉小米14,一款集设计与性能于一身的手机，给你带...",内容基础上增加一些表情符号、网络用语等等来增加文案的活力,"标题: 小米13大战小米14,究竟哪个性价比更高？🔥\n内容:🔥小米13大战小米14,究竟哪...",增加一些工作沟通的使用场景描述,"内容:🔥🔥小米13大战小米14,究竟哪个性价比更高？🔥🔥\n🎯🎯小米13和小米14,究竟哪个...",增加一些卖点介绍：4k屏幕分辨率、高通处理器、5G、无线充电,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,请你帮忙创作小红书文风的爆款内容推荐一下小米13 Pro手机,标题: 小米13pro 真的可以！💥💥💥\n内容:🎉【小米13 Pro 】首发党再次成功！🎉...,增加一些卖点介绍，比如机身重量198g、高刷新率、多核心处理器、面容识别等等,标题:🌟【小米13 Pro 】首发党再次成功！🌟\n内容:🎉【小米13 Pro 】首发党再次...,站在时尚潮流的角度来推荐产品,标题:🌟【小米13 Pro 】首发党再次成功！🌟\n内容:🎉【小米13 Pro 】首发党再次...,文案内容增加一些故事性描述来激发读者共鸣,"标题:💥小米13 Pro: 首发党的胜利，性能与美貌并存！💥\n内容:🎉小米13 Pro,这...",将产品推荐给差旅人士,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,写一篇介绍小米13手机的文案，要符合当下小红书爆款风格,"标题:🌟小米13: 性能与美学的完美融合，让你一见倾心！📱\n内容:🎉小米13,一款让你一见...",站在好物推荐的角度来推荐产品,"标题:🌟小米13: 性能与美学的完美融合，让你一见倾心！📱\n内容:🎉小米13,一款让你一见...",增加一些卖点介绍，如无线充电器、type-接口、骁龙处理器、蓝牙4.0等,标题:🌟小米13: 性能与美学的完美融合，让你一见倾心！📱\n内容:🎉小米13手机，一款让你...,增加一些看电影的使用场景描述,标题:🌟小米13: 性能与美学的完美融合，让你一见倾心！📱\n内容:🎉小米13手机，一款让你...,把产品推荐给宝妈们,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,请写一篇小红书文风的爆款文案，推荐产品华为Mate 50 RS手机,标题: 华为 Mate 50 RS: 通信与时尚的完美融合📱✨\n内容:🎉【华为 Mate ...,增加一些交流学习的使用场景描述,标题: 华为 Mate 50 RS: 探索通信新境界，尽享科技魅力🌟🚀\n内容:🚀华为 Ma...,站在日常生活的角度，用第一人称口吻来推荐产品,标题: 华为 Mate 50 RS: 探索通信新境界，尽享科技魅力🌟🚀\n内容:🚀华为 Ma...,增加一些该产品的卖点介绍，如鸿蒙系统、不锈钢、蓝牙4.0、国产手机等等,标题: 华为 Mate 50 RS: 探索通信新境界，尽享科技魅力🌟🚀\n内容:🌟🌟🌟华为 ...,文案内容增加一些故事性描述来激发读者共鸣,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,请生成一篇关于推荐华为Mate 50 Pro+手机的小红书爆款文案,标题: 华为 Mate50Pro+: 全能旗舰，引领时尚潮流！\n内容: 大家好，今天给大家...,站在个性潮玩的角度来推荐产品,标题: 华为 Mate50Pro: 全能旗舰，引领时尚潮流！\n内容: 大家好呀！今天给大家...,增加一些该产品的卖点介绍，比如鸿蒙、5G手机、60hz刷新率、简化包装,标题: 华为 Mate50Pro: 全能旗舰，引领时尚潮流！\n内容: 大家好呀！今天给大家...,在上轮内容基础上使用一些表情符号、网络用语等等来增加文案的活力,内容: 大家好呀！今天给大家推荐的是一款非常优秀的手机，那就是华为 Mate50Pro!🎉💥...,把产品推荐给手机发烧友们,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,帮我写一篇小红书爆款文案推荐一下华为Mate 50手机,标题: 华为 Mate50: 遥遥领先，绝绝子啊！\n内容: 嗨宝贝们！👋今天给大家推荐的是...,增加一些该产品的卖点介绍，比如4K分辨率、新款手机、机身198g、广角摄像头等等,标题: 华为 Mate50: 遥遥领先，绝绝子啊！🌟📱\n内容: 大家好呀！今天给大家推荐的...,站在数码科技的角度来推荐产品,标题: 华为 Mate50: 遥遥领先，科技与美学的完美融合！🌟📱\n内容: 大家好呀！今天...,增加一些休闲娱乐的使用场景描述,标题: 华为 Mate50: 遥遥领先，科技与美学的完美融合！🌟📱\n内容: 大家好呀！今天...,在上轮内容基础上增加一些表情符号、网络用语等等来增加文案的活力,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,我需要写一篇符合当下小红书爆款风格的文案介绍华为P60 Ultra手机,标题:🌟📱【华为 P60 Ultra 】华为旗舰新篇章，引领科技潮流！💥💥💥\n内容:🎉【华...,增加一些通勤支付的使用场景描述,标题:🌟📱【华为 P60 Ultra 】华为旗舰新篇章，引领科技潮流！💥💥💥\n内容:🎉【华...,在上轮内容基础上增加一些表情符号来增加文案的活力,标题:🌟📱【华为 P60 Ultra 】华为旗舰新篇章，引领科技潮流！💥💥💥\n内容: 大家...,增加一些该产品的卖点介绍，比如独立包装、harmony、2.4ghz、磨砂材质,标题:🌟📱【华为 P60 Ultra 】华为旗舰新篇章，引领科技潮流！💥💥💥\n内容: 大家...,在上轮文案内容上增加一些故事性描述来激发读者共鸣,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,请帮我写一篇符合当下小红书爆款风格的文案介绍华为P60 Pro+手机,标题:🌟华为 P60 Pro+: 引领摄影新潮流，赋予你无限创作可能📸🌟\n内容:✨哈喽，亲...,在上轮内容基础上增加一些表情符号、网络用语等等来增加文案的活力,标题:🌟华为 P60 Pro+: 引领摄影新潮流，赋予你无限创作可能📸🌟\n内容:Hey 亲...,增加一些玩游戏的使用场景描述,标题:🎮华为 P60 Pro+: 游戏玩家的最佳伙伴，让你畅享极致游戏体验！🎮🌟\n内容:H...,请将产品推荐给上班群体,标题:🎮华为 P60 Pro+: 游戏玩家的最佳拍档，摄影达人必备！🎮🌟\n内容:Hey 亲...,增加一些该产品的卖点介绍，比如5G手机、旗舰机、陶瓷材质、多摄像头,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
for i in range(1):
    metic =  [0] * 14 
    request = ""
    comment = ""
    for j in range(10):
        copywrite = str(df.iloc[i,2*j+2])
        count = 0
        if copywrite != "nan":
            request += (str(df.iloc[i,2*j+1])+'/n')
            prompt_user="写作要求:\n   " + request +" \n营销文案:\n   "+ copywrite
            print(i,j)
            print(prompt_user)
            success = 0
            while(success == 0):
                try:
                    result = get_completion(prompt_sys=prompt_sys,prompt_user=prompt_user)
                    print(result)
                    result = json.loads(result)
                    a = 0
                    for item in result:
                        metic[a] += int(result[item][0])
                        a += 1
                    comment += result['受众群体'][1]+';'+result['故事性描述'][1]+';'+';'+result['文本创作'][1]+';'+result['安全能力'][1]+'。\n'
                    print(metic)
                    print(comment)
                    count += 1
                    success = 1
                except:
                    print("响应时间过长")
    for b in range(14):
        df.iloc[i,b+23] = metic[b]/count
    summary_prompt = """
    帮我总结以下内容：

    """
    summary = get_answer(summary_prompt + comment)
    df.iloc[i,21] = summary
    df.iloc[i,22] = sum(metic)/len(metic)

0 0
写作要求:
   你是一位小红书爆款写作专家，请你创作一篇小红书文风的爆款文案推荐一下小米13 Ultra，来吸引潜在客户购买/n 
营销文案:
   标题:💥小米13 Ultra: 颠覆你的视觉体验，让你心动不已！🌟
内容:🔥想要一款既能满足你对性能的极致追求，又能展现你独特品味的手机吗？小米13 Ultra 绝对是你不可错过的选择！🚀
🌟小米13 Ultra 搭载了强大的处理器，让你无论是运行大型游戏还是处理复杂任务都能游刃有余。🎮
🌟它还拥有令人惊艳的屏幕，色彩鲜艳、清晰度极高，让你沉浸在每一个细节之中。📱
🌟更令人惊喜的是，这款手机的外观设计也十分出色，无论是手感还是观感都达到了新的高度。🌟
🌟如果你是一个对手机性能和外观都有高要求的用户，那么小米13 Ultra 绝对是你不可错过的选择！🚀
🔥赶紧入手一台小米13 Ultra,感受它带来的极致体验吧！🔥
#小米13Ultra #性能强大#屏幕惊艳#外观设计#手机推荐
{
    "平台调性":["5","文案中大量使用了小红书用户喜爱的表情符号和热情的语气，符合小红书的年轻、活泼和具有情感表达的平台调性。"],
    "品牌信息":["5","品牌信息非常清晰，文案明确提到小米13 Ultra，并围绕这款产品进行了详细的描述。"],
    "品牌特点/卖点":["5","文案成功突出了小米13 Ultra的几个主要特点：性能强大、屏幕出色、外观设计感好，这与营销要求保持一致，连接产品卖点与消费者需求。"],
    "受众群体":["5","通过强调手机的性能、屏幕和外观设计，文案精准地吸引了对手机性能和外型有高要求的潜在客户群体。"],
    "可控字数":["5","文案长度适中，适合小红书的浏览习惯，符合社交媒体的文案长度要求。"],
    "设定发文者视角":["5","发文者的视角清晰，就是一位熟悉小红书并且了解小米13 Ultra的专家所写的推荐文案。"],
    "文案标题":["5","标题引人注目，使用了表情符号和强调词汇，能有效吸引目标受众的注意力。"],
    "故事性描述":["3","文案没有用故事性的描述，而是直接介绍了产品特点，这是小红书用户可能期待的形式，但加入用户使用场景的小故事会更具吸引力。"],
    "表情/网络用语":["5","充分使用了表情

APIConnectionError: Connection error.